# <center>Natural Language Processing Using NLTK (II)</center>

References:
 - http://www.nltk.org/book_1ed/
 - https://web.stanford.edu/class/cs124/lec/Information_Extraction_and_Named_Entity_Recognition.pdf
 - https://nlpforhackers.io/complete-guide-to-spacy/
 - https://radimrehurek.com/gensim/models/phrases.html

## 2. NLP Objectives and Basic Steps

 - Objectives:
   * Split documents into words, punctuation sysmbols, or segments
   * Understand vocabulary of the text
   * Extract features for further text mining tasks
 - Basic processing steps:
   * Tokenization: split documents into individual words and punctuation symbols
   * Remove stop words and filter tokens
   * **POS (part of speech) Tagging**  
   * **Normalization: Stemming, Lemmatization**
   * **Named Entity Recognition (NER)**
   * **Term Frequency and Inverse Dcoument Frequency (TF-IDF)**
   * **Create document-to-term matrix (bag of words)**

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import nltk

# Sample text for analysis

news=["Oil prices soar to all-time record", 
"Stocks end up near year end", 
"Money funds rose in latest week",
"Stocks up; traders eye crude oil prices",
"Dollar rising broadly on record trade gain"]
text=". ".join(news).lower()
text

## 4. POS (Part of Speech) Tagging

 - What is POS Tagging:
   * The process of marking up a word in a text as corresponding to a particular part of speech (e.g. nouns, verbs, adjectives, adverbs etc.), based on both **its definition**, as well as its **context** — adjacent and related words in a phrase, sentence, or paragraph. 
 - Why POS Tagging: 
   * **disambiguation**: A word may have different meanings. POS tag is a potential strong signal for word sense disambiguation. For example, "I fish a fish"
   * **Phrase extraction**: Use POS rules to define accepted phrases (or information unit), or collocations for indexing and retrieval:
     * Adj + Noun, e.g. nice house
     * Verb + Noun, e.g. play football
     * typical collocation patterns (https://nlp.stanford.edu/fsnlp/promo/colloc.pdf):
       - Adj + Noun: e.g. linear function
       - Noun + Noun: e.g. regression coefficient
       - Adj + Adj + Noun: e.g. Gaussian random variable
       - Noun + Adj + Noun: e.g. mean squared error
       - Noun + Noun + Noun: e.g. class probability function
       - Noun + Preposition + Noun: e.g. dregrees of freedom
   * **Filter tokens**:  some POS have less importance in retrieval, e.g. stopwords such as ‘a’, ‘an’, ‘the’, and other glue words like 'in', 'on', 'of' etc.
   * Find other forms of a word based on POS
        * Noun: plural and singular
        * Verb: past, present and future tense
        * Adjective: positive, comparative, and superlative
 - List of Penn Treebank Tags can be found at https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
 - A tagger (program for tagging) is trained based on a corpus using machine learning approaches. It may not be very accurate when applying it your corpus.
   - Stanford tagger (~97%)
   - NLTK default tagger (PerceptronTagger)
- How to train a tagger?
    - *Stochastic/Probabilistic Methods*: Automated ways to assign a PoS to a word based on the probability that a word belongs to a particular tag or based on the probability of a word being a tag based on a sequence of preceding/succeeding words.
    - Algorithms: SVM, Naive Bayes, CRF (conditional random fields), Hidden Markov Models (HMM)
    - Features: 
        - **Word**: the word itself. Some words are always one PoS, others not.
        - **is_first**, is_last: check if it is the first or last in the sentence.
        - **is_capitalized**: first letter is caps? Maybe it is a proper noun...
        - **is_all_caps** or *is_all_lower*: checks for acronyms (or common words).
        - **prefixes/suffixes**: check word initialization/termination
        - **prev_word/next_word**: checks the preceding and succeding word.
        - **has-hyphen**: words with '-' may be adjectives.
        - **is_numeric**: for numbers.
        - **capitals_inside**: weird cases. Maybe nouns.
   - Check https://colab.research.google.com/drive/1d7LO_0665DYw6DrVJXXautJAJzHHqYOm#scrollTo=bfD5ujGijuUF for model training

In [ ]:
# Exercise 4.1. To find all tags in treebank
#nltk.help.upenn_tagset()

# find the meaning of a specific tag
nltk.help.upenn_tagset('JJ')


In [ ]:
# Exercise 4.2. NLTK POS Tagging

# The input to the tagging function is a list of words

# tokenize the text
tokens=nltk.word_tokenize(text)

# tag each tokenized word
tagged_tokens= nltk.pos_tag(tokens)

tagged_tokens
 

In [ ]:
# Exercise 4.3. Extract Phrases by POS

# Extract phrases in pattern of adjective + noun
# i.e. nice house, growing market

bigrams=list(nltk.bigrams(tagged_tokens))
#print(bigrams)

phrases=[ (x[0],y[0]) for (x,y) in bigrams \
         if x[1].startswith('JJ') \
         and y[1].startswith('NN')]

print(phrases)

In [ ]:
# Exercise 4.4. Extract Noun+Verb, 
# i.e. prices soar

phrases=[ (x[0],y[0]) for (x,y) in bigrams \
         if x[1].startswith('NN') \
         and y[1].startswith('VB')]

print(phrases)


## 5. Normalization: Stemming & Lemmatization
 - What is normalization:
   - Converts a list of words in **different surface forms** to a more **uniform form**, e.g.
        * a word with different forms, e.g. organize, organizes, organized, and organizing
        * families of derivationally related words with similar meanings, such as democracy, democratic, and democratization.
 - Why normalization
   - **improve text matching**: in many situations, it seems as if it would be useful for a search for one of these words to return documents that contain another word in the set.
   - reduce featue space generated from text
 - Stemming and lemmatization are two common techinques


### 5.1. Stemming 

* **Stemming**: reducing inflected (or sometimes derived) words to their **stem, base or root** form. 
   * For example, **crying** -> **cri**. 
   * Stemming may not generate a real word, but a root form. 
   * The stemming program is called stemmer. 
       * Famous stemers are Porter stemmer, Lancaster Stemmer, Snowball Stemmer.

In [ ]:
# Exercise 5.1.1. Stermming Using Porter Stemmer

from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

print("Stem of organizing/organized/organizes/organization")
print(porter_stemmer.stem('organizing'))
print(porter_stemmer.stem('organized'))
print(porter_stemmer.stem('organizes'))
print(porter_stemmer.stem('organization'))

print("\nStem of crying")
print(porter_stemmer.stem('crying'))

### 5.2. Lemmatization

* **Lemmatization**: determining the lemma for a given word, 
   * A lemma is a word which stands at the head of a definition in a dictionary, e.g. run (lemma),  runs, ran and running (inflections) 
   * Lemmatization is a complex task involving understanding context and determining the part of speech of a word in a sentence 
      * e.g. "organized" (verb or adjective?)
   * The widely used Lemmatization method is based on WordNet, a large lexical database of English.

* **Difference** between stemming and lemmatization: 
   * a stemmer operates on a single word **without knowledge of the context**, and therefore cannot discriminate between words which have different meanings depending on part of speech. While, lemmatization **requires context and POS tags**. 
   * Stemming may not generate a real word, but lemmization always generates real words.
   *  However, stemmers are typically easier to implement and run faster with reduced accuracy.

In [ ]:
# Exercise 5.2.1. Lemmatization

# wordnet lemmatizer takes POS tag as a parameter
# However, wordnet has its own tag set, 
# different from treebank tag set
# The default POS tag is noun 

from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet

wordnet_lemmatizer = WordNetLemmatizer()

print("organizing (verb) ->", \
      wordnet_lemmatizer.lemmatize\
      ('organizing', wordnet.VERB))
print('organized (verb) ->', \
      wordnet_lemmatizer.lemmatize\
      ('organized', wordnet.VERB))
print('organized (adjective) ->',\
      wordnet_lemmatizer.lemmatize('organized', \
                                   wordnet.ADJ))
print('organization (noun) ->',\
      wordnet_lemmatizer.lemmatize('organization'))
print('crying (adjective) ->',\
      wordnet_lemmatizer.lemmatize('crying', \
                                   wordnet.ADJ))
print('crying (verb) ->', \
      wordnet_lemmatizer.lemmatize('crying', \
                                   wordnet.VERB))

# compare the result with Exercise 5.1.1.

## 6. Named Entity Recognition (NER)

- Definition: find and classify real word entities (Person, Organization, Event etc.) in text
- Example: sentence "Jim bought 300 shares of Acme Corp. in 2006" can be annotated as "**[Jim]<sub>Person</sub>** bought 300 shares of **[Acme Corp.]<sub>Organization</sub>** in 2006"
- Uses of NER:
   *  Information Extraction: extract clear, factual information, i.e., Who did what to whom when?
   *  Named entities can be indexed, and their relations can be extracted.
   *  Sentiment can be attributed to companies or products
   *  For question answering, answers are often named entities.
- Techniques for NER
   * Regular expression: Telephone numbers, emails, Capital names (e.g. Capitalized word + {city,  center, river}
      * Adantages: simple and sometimes effective
      * Disadvantage: 
         * first word of a sentence is capitalized; sometimes, titles are all capitalized; new proper names constantly emerges (e.g. movie titles, books, etc.)
         * proper names may be ambiguous, e.g. Jordan can be *person* or *location*
   * Supervised learning (IOB) (https://arxiv.org/abs/cmp-lg/9505040)
       1. Collect a set of representative training documents
       2. Label each token for its entity class (I: inside entity, B: begining entity) or other (O)
       3. Design feature extractors appropriate to the text and classes, e.g. current word, pre/next word, pos tags etc.
       4. Train a sequence classifier to predict the labels from the data

In [ ]:
# Exercise 6.1. Use NLTK for Named Entity Recognition

from nltk import word_tokenize, pos_tag, ne_chunk, Tree

sentence = "Jim bought 300 shares of Acme Corp. in 2006."

# the input to ne_chunk is list of (token, pos tag) tuples
ner_tree=ne_chunk(pos_tag(word_tokenize(sentence)))

# ne_chunk returns a tree
# print the tree
Tree.fromstring(str(ner_tree)).pretty_print()


# get PERSON out of the tree
person=[]
for t in ner_tree.subtrees():
    if t.label() == 'PERSON':
        person.append(t.leaves())
print("PERSON",person)


# how to extract organization?

## 7. Term Frequency and Inverse Dcoument Frequency (TF-IDF)
 - Motivation: How to identify important words (or phrases, named entities) in a text in a collecton or corpus? When search for documents, we'd like to have these important words are matched.
 - Intuition: 
   * In a document, if a word/term/phrase is repeated many times, it is likely important. 
   * However, if it appears in most of the documents in the corpus, then it has little discriminating power in determining relevance. 
   * For instance, a collection of documents on the auto industry is likely to have the term auto in almost every document. Search by "auto" you may get all the documents. 
 - **TF-IDF**: is composed by two terms: 
      - `TF (Term Frequency)`: which measures how frequently a term, say w, occurs in a document. 
      - `IDF (Inverse Document Frequency)`: measures how important a term is within the corpus. 
 
 - TF-IDF provides another way to remove stop words

### 7.1. Term Frequency (TF)
- Measures how frequently a term, say w, occurs in a document, say $d$. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. 
- Thus, the frequency of $w$ in $d$, denoted as $freq(w,d)$ is often divided by the document length (a.k.a. the total number of terms in the document, denoted as $|d|$) as a way of normalization: $$tf(w,d) = \frac{freq(w,d)}{|d|}$$
- Example: d="Stocks end up near year end"
   * `tf('Stocks',d)=?`
   * `tf('end',d)=?`

### 7.2. Inverse Document Frequency (IDF)
- Measures how important a term is within the corpus. 
- However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. 
- Thus we need to weigh down the frequent terms while scale up the rare ones. 
- Let $|D|$ denote the number of documents, $df(w,D)$ denotes the number of documents with term $w$ in them. Then, $$idf(w) = ln(\frac{|D|}{df(w,D)})+1$$ Or a smoothed version: $$idf(w) = ln(\frac{|D|+1}{df(w,D)+1})+1$$
- Examples: 
  * Considering dataset:
       1. "Oil prices soar to all-time record", 
       2. "Stocks end up near year end", 
       3. "Money funds rose in latest week",
       4. "Stocks up; traders eye crude oil prices",
       5. "Dollar rising broadly on record trade gain"
  * `idf('Stocks')=`?
  * `idf('all-time')=`?
  * Discussion:
     * What words get very low IDF score?
     * What words get very high IDF score?


### 7.3. TF-IDF 
- Let $s(w,d)=tf(w,d) * idf(w)$, normalize the TF-IDF score of each word in a document normalized by the Euclidean norm, then 
   $$tfidf(w,d)=\frac{s(w,d)}{\sqrt{\sum_{w \in d}{s(w,d)^2}}}$$

In [ ]:
# Exercise 7.1. computing tf-idf


import nltk, re, string
from nltk.corpus import stopwords

# library for normalization
from sklearn.preprocessing import normalize

# numpy is the package for matrix caculation
import numpy as np  

stop_words = stopwords.words('english')

docs=["Oil prices soar to all-time record", 
"Stocks end up near year end", 
"Money funds rose in latest week",
"Stocks up; traders eye crude oil prices",
"Dollar rising broadly on record trade gain"]   


In [ ]:
# Step 1. get tokens of each document as list

def get_doc_tokens(doc):
    tokens=[token.strip() \
            for token in nltk.word_tokenize(doc.lower()) \
            if token.strip() not in stop_words and\
               token.strip() not in string.punctuation]
    
    # you can add bigrams, collocations, or lemmatization here
    
    # create token count dictionary
    token_count=nltk.FreqDist(tokens)
    
    # or you can create dictionary by yourself
    #token_count={token:tokens.count(token) for token in set(tokens)}
    return token_count

# step 2. process all documents to 
# a dictionary of dictionaries
docs_tokens={idx:get_doc_tokens(doc) \
             for idx,doc in enumerate(docs)}
docs_tokens

In [ ]:
# step 3. get document-term matrix
# contruct a document-term matrix where 
# each row is a doc 
# each column is a token
# and the value is the frequency of the token

import pandas as pd

# since we have a small corpus, we can use dataframe 
# to get document-term matrix
# but don't use this when you have a large corpus

dtm=pd.DataFrame.from_dict(docs_tokens, \
                           orient="index" )
dtm
dtm=dtm.fillna(0)
dtm

# sort by index (i.e. doc id)
dtm = dtm.sort_index(axis = 0)
dtm

In [ ]:
# step 4. get normalized term frequency (tf) matrix

# convert dtm to numpy arrays
tf=dtm.values

# sum the value of each row
doc_len=tf.sum(axis=1)
doc_len

# divide dtm matrix by the doc length matrix
tf=np.divide(tf, doc_len[:,None])

# set float precision to print nicely
np.set_printoptions(precision=2)

tf

In [ ]:
# step 5. get idf

# get document freqent
df=np.where(tf>0,1,0)
#df

# get idf
idf=np.log(np.divide(len(docs), \
        np.sum(df, axis=0)))+1
print("\nIDF Matrix")
idf

# what is the size of idf array?

smoothed_idf=np.log(np.divide(len(docs)+1, np.sum(df, axis=0)+1))+1
print("\nSmoothed IDF Matrix")
smoothed_idf



In [ ]:
# step 6. get tf-idf
print("TF-IDF Matrix")
s = tf*idf
s

tf_idf=normalize(tf*idf)   # is broadcast possible here?
tf_idf

print("\nSmoothed TF-IDF Matrix")
smoothed_tf_idf=normalize(tf*smoothed_idf)
smoothed_tf_idf


- TF-IDF matrix gives **weight** of each word in each document
- Documents:
    1. "Oil prices soar to all-time record", 
    2. "Stocks end up near year end", 
    3. "Money funds rose in latest week",
    4. "Stocks up; traders eye crude oil prices",
    5. "Dollar rising broadly on record trade gain"

In [ ]:
# For better visualization, let's make the tf-idf array a dataframe
pd.options.display.float_format = '{:,.2f}'.format # set format for float

pd.DataFrame(smoothed_tf_idf, columns = dtm.columns)
# the dtm dataframe we created in Step 3 has each word as a column

In [ ]:
# Exercise 7.2. Find the top three words 
# of each document by TF-IDF weight

top=smoothed_tf_idf.argsort(axis = 1)[:,::-1][:,0:3]
top

for row in top:
    print([dtm.columns[x] for x in row])

### 7.4. What to do with TF-IDF
- This is the `feature sapce` of text mining (a.k.a. `Bag of Words` or `Vector Space Model`)
- Identify important words in each document
- Find similar documents
    * How to measure simialrity (or distance)? http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.332.4480&rep=rep1&type=pdf
        - `Euclidean distance`
        - `Cosine distance`
    * Euclidean distance:
        - It can be **large** for vectors of high dimension 
        - `Curse of dimensionality`: In a high-dimensional space, the ratio between the nearest and farthest points approaches 1, i.e. the points essentially become uniformly distant from each other. (https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)
    * Cosine similarity: The similarity between two documents is a function of the angle between their vectors in the if-idf vector space. 
      <img src='cosine.png' width=50% />
      <img src='cosine_formula.svg' width=50% />
      - Example: A=[0,2,1], B=[1,1,2], then
      $$cosine(A,B)=\frac{0*1+2*1+1*2}{\sqrt{0+4+1}*\sqrt{1+1+4}}$$

In [ ]:
## Exercise 7.4.1 Document similarity

# package to calculate distance
from sklearn.metrics import pairwise_distances

# calculate cosince distance of every pair of documents 
# similarity is 1-distance
similarity=1-pairwise_distances(tf_idf, metric = 'cosine')
similarity

# find top doc similar to the first one
# Note the diagonal value is 1, which is the largest

np.argsort(similarity)[:,::-1][0,0:2]

for idx, doc in enumerate(docs):
    print(idx,doc)

### 7.5. Put Everyting together -- Computing TF-IDF

In [ ]:
import nltk, re, string
from sklearn.preprocessing import normalize
from nltk.corpus import stopwords
# numpy is the package for matrix cacluation
import numpy as np  
import pandas as pd

stop_words = stopwords.words('english')

# Step 1. get tokens of each document as list
def get_doc_tokens(doc):
    tokens=[token.strip() \
            for token in nltk.word_tokenize(doc.lower()) \
            if token.strip() not in stop_words and\
               token.strip() not in string.punctuation]
    
    # you can add bigrams, collocations, stemming, 
    # or lemmatization here
    
    token_count={token:tokens.count(token) for token in set(tokens)}
    return token_count

def tfidf(docs):
    # step 2. process all documents to get list of token list
    docs_tokens={idx:get_doc_tokens(doc) \
             for idx,doc in enumerate(docs)}

    # step 3. get document-term matrix
    dtm=pd.DataFrame.from_dict(docs_tokens, orient="index" )
    dtm=dtm.fillna(0)
    dtm = dtm.sort_index(axis = 0)
      
    # step 4. get normalized term frequency (tf) matrix        
    tf=dtm.values
    doc_len=tf.sum(axis=1, keepdims=True)
    tf=np.divide(tf, doc_len)
    
    # step 5. get idf
    df=np.where(tf>0,1,0)
    #idf=np.log(np.divide(len(docs), \
    #    np.sum(df, axis=0)))+1

    smoothed_idf=np.log(np.divide(len(docs)+1, np.sum(df, axis=0)+1))+1    
    smoothed_tf_idf=tf*smoothed_idf
    
    return smoothed_tf_idf

## 8. Keyowrd/Keyphrase Extraction

Why keywords/keyphrses (Wang and Xiao 2008)
- A keyphrase is defined as a meaningful and significant expression consisting of one or more words in a document. 
- Appropriate keyphrases can serve as a highly condensed summary for a document, and they can be used as a label for the document to supplement or replace the title or summary, or they can be highlighted within the body of the document to facilitate users’ fast browsing and reading. 
- Moreover, document keyphrases have been successfully used in the following IR and NLP tasks: document indexing, document classification, document clustering, and document summarization.

References: 
- Mihalcea, R., & Tarau, P. (2004, July). TextRank: Bringing order into texts. Association for Computational Linguistics.
- Wan, Xiaojun and Jianguo Xiao. 2008. Single document keyphrase extraction using neighborhood knowledge. In Proceedings of the 23rd AAAI Conference on Artificial Intelligence, pages 855–860.
- Florescu, C. and Cornelia, C. (2017). PositionRank: An Unsupervised Approach to Keyphrase Extraction from Scholarly Documents. In proceedings of ACL*, pages 1105-1115

A nice tutorial: <a href='https://towardsdatascience.com/textrank-for-keyword-extraction-by-python-c0bae21bcec0'> Understand TextRank for Keyword Extraction by Python </a>
   - Concept Map example:

<img src='https://miro.medium.com/max/1400/1*JeYlqJTA5MpKsaWwHkhKQw.jpeg' width='70%'>

### 8.1 Textrank

TextRank is an algorithm based on PageRank, which often used in keyword extraction and text summarization. 

**Pagerank** 

- PageRank is used primarily for ranking web pages in online search results 
- All web pages as a big directed graph, where a node is a webpage and each edge is a link from the source page to the target page. 
- For example, if webpage A has the link to web page B, it can be represented as a directed edge from A to B.
- <a href= "https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/"> An illustrated graph </a>
<img src = 'https://cdn.analyticsvidhya.com/wp-content/uploads/2018/10/Pagerank11.png' width="60%">

- Pagerank formula

    - <img src='https://miro.medium.com/max/1400/1*hheHfLOTjPW3uSsSxWKylQ.png'>

- Example
<img src ='https://miro.medium.com/max/1030/1*DkZjlRNEjPSc8RNL7yWggA.jpeg' width = '60%'>
    
    
Initial transition matrix: each column indicates the transition probability from one node to others, e.g. p(e|a) = 1.


Let's focus on nodes `a, b, e, f`. The transition matrix is shown below:    
<img src='https://miro.medium.com/max/1400/1*ppIReXWpavbuvAHfi08EGw.png' width="80%">


Assuming initial weight `V` for each node is 1 and `d = 0.85`. We can update `V` iteratively until `V` converges.
- For example, $V(e) = (1-0.85) + 0.85*(1 + \frac{1}{2} * 1)$

### Textrank: rank words within a document

- We consider text within a specific window as a chunk
- Formulate each word as a node
- Chunks are used as edges to connect nodes. The weight on the edge between each pair of nodes denotes the number of chunks sharing the same pair of words
- Then we follow pagerank to rank the importance of each node

- Example sentence:  `The Wandering Earth feels like a throwback to a few familiar eras of American filmmaking.`
    - After removing punctuations and stopping words, we'll have `[Wandering, Earth, feels, throwback, familiar, eras, American, filmmaking]`
    - Consider window = 3, we'll have chunks `[Wandering, Earth, feels], [Earth, feels, throwback], ...`
    - Formulate the graph as follows:
        - Each word is a node
        - The edge weight between two nodes is the times they show in trunks (edge weight can also be binarized). 
    - Calculate textrank for each word
    - Rank each word by their text rank



- A reference implementation (shown below) can be found at https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/.
- This algorithm is also implemented at `Textacy` package: https://textacy.readthedocs.io/en/0.11.0/api_reference/extract.html#module-textacy.extract.keyterms.textrank